In [61]:
#model uses 2024 GEBCO bathymetry file downloaded in the repo 

In [1]:
import Pkg
Pkg.activate(@__DIR__)  
#Pkg.instantiate()        # all dependencies are installed -- do once only

  Activating project at `~/Desktop/scripts/oceananigans-tutorial`


In [6]:
#in case activate environment does not work
using Pkg
using Oceananigans
using Oceananigans.Units
import ClimaOcean
using Printf
using Dates, CFTime
Pkg.add("Interpolations")
Pkg.add("NCDatasets")
Pkg.add("CairoMakie")

[ Info: Precompiling Oceananigans [9e8cae18-63c1-5223-a75c-80ca9d6e9a09] (cache misses: wrong dep version loaded (4), dep missing source (2))
[ Info: Precompiling ClimaOcean [0376089a-ecfe-4b0e-a64f-9c555d74d754] (cache misses: wrong dep version loaded (4))
   Resolving package versions...
  No Changes to `~/Desktop/scripts/oceananigans-tutorial/Project.toml`
  No Changes to `~/Desktop/scripts/oceananigans-tutorial/Manifest.toml`
Precompiling project...
   2265.0 ms  ✓ Gettext_jll
    401.0 ms  ✓ XSLT_jll
   3101.3 ms  ✓ NetCDF
   3585.5 ms  ✓ HypergeometricFunctions
   3929.2 ms  ✓ SpecialFunctions → SpecialFunctionsChainRulesCoreExt
   5656.7 ms  ✓ IntervalArithmetic → IntervalArithmeticForwardDiffExt
   2195.3 ms  ✓ LoopVectorization → ForwardDiffExt
  12233.7 ms  ✓ Fetch
  13319.4 ms  ✓ Roots → RootsForwardDiffExt
    804.4 ms  ✓ Glib_jll
   1295.3 ms  ✓ StatsFuns
   1027.2 ms  ✓ Qt6Base_jll
    497.5 ms  ✓ Cairo_jll
    729.3 ms  ✓ StatsFuns → StatsFunsInverseFunctionsExt
   1010.

In [2]:
using NCDatasets
p = "filepath_of_grid" 

[ Info: Precompiling NCDatasets [85f8d34a-cbdd-5861-8df4-14fed0d494ab] (cache misses: wrong dep version loaded (2))


"/Users/suhanidalal/Desktop/scripts/oceananigans-tutorial/GEBCO_18_Feb_2025_6614cd9c6dc1/gebco_2024_n50.0_s47.0_w-127.0_e-120.0.nc"

In [3]:
#using the netcdf
ds = NCDataset(p)

println(ds)
println("Variables in dataset: ", keys(ds))
println("Dimensions: ", ds.dim)

lon = ds["lon"][:]  
lat = ds["lat"][:]  
depth = ds["elevation"][:, :] 
lat_res = abs(lat[2] - lat[1])  
lon_res = abs(lon[2] - lon[1])  

println("lat Resolution: ", lat_res, " degrees")
println("long Resolution: ", lon_res, " degrees")

close(ds)
#500 m resolution 

Dataset: /Users/suhanidalal/Desktop/scripts/oceananigans-tutorial/GEBCO_18_Feb_2025_6614cd9c6dc1/gebco_2024_n50.0_s47.0_w-127.0_e-120.0.nc
Group: /

Dimensions
   lat = 720
   lon = 1680

Variables
  lat   (720)
    Datatype:    Float64 (Float64)
    Dimensions:  lat
    Attributes:
     standard_name        = latitude
     long_name            = latitude
     units                = degrees_north
     axis                 = Y
     sdn_parameter_urn    = SDN:P01::ALATZZ01
     sdn_parameter_name   = Latitude north
     sdn_uom_urn          = SDN:P06::DEGN
     sdn_uom_name         = Degrees north

  lon   (1680)
    Datatype:    Float64 (Float64)
    Dimensions:  lon
    Attributes:
     standard_name        = longitude
     long_name            = longitude
     units                = degrees_east
     axis                 = X
     sdn_parameter_urn    = SDN:P01::ALONZZ01
     sdn_parameter_name   = Longitude east
     sdn_uom_urn          = SDN:P06::DEGE
     sdn_uom_name         = Deg

closed Dataset

In [7]:
Nx = length(lon) 
Ny = length(lat)
Nz = 25
long_min = (minimum(lon))
long_max = (maximum(lon))
lat_min = (minimum(lat))
lat_max = (maximum(lat))
min_z = (minimum(depth))
arch = CPU() #change to GPU 

grid = LatitudeLongitudeGrid(arch,
                             size = (length(lon), length(lat), 25), 
                             halo = (7, 7, 7),  
                             longitude = (minimum(lon), maximum(lon)),
                             latitude = (minimum(lat), maximum(lat)),
                             z = (minimum(depth), 0))

1680×720×25 LatitudeLongitudeGrid{Float64, Bounded, Bounded, Bounded} on CPU with 7×7×7 halo and with precomputed metrics
├── longitude: Bounded  λ ∈ [-126.998, -120.002] regularly spaced with Δλ=0.00416419
├── latitude:  Bounded  φ ∈ [47.0021, 49.9979]   regularly spaced with Δφ=0.00416088
└── z:         Bounded  z ∈ [-2643.0, 0.0]       regularly spaced with Δz=105.72

In [8]:
# Pkg.add("Interpolations") #if needed

using Interpolations

bathymetry_interp = interpolate((lon, lat), depth, Gridded(Linear()))

bottom_height = [bathymetry_interp(lon[i], lat[j]) for i in 1:length(lon), j in 1:length(lat)]

grid = ImmersedBoundaryGrid(grid, GridFittedBottom(bottom_height))


   Resolving package versions...
  No Changes to `~/Desktop/scripts/oceananigans-tutorial/Project.toml`
  No Changes to `~/Desktop/scripts/oceananigans-tutorial/Manifest.toml`
[ Info: Precompiling ChainRulesCoreExt [eae2faf6-b232-58cb-a410-7764fda2830c] (cache misses: wrong dep version loaded (6))
┌ Warning: Module CUDA with build ID fafbfcfd-7ec3-44b1-0003-28f3c7f15bea is missing from the cache.
│ This may mean CUDA [052768ef-5323-5732-b1bb-66c8b64840ba] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:2541
┌ Info: Skipping precompilation due to precompilable error. Importing ChainRulesCoreExt [eae2faf6-b232-58cb-a410-7764fda2830c].
└   exception = Error when precompiling module, potentially caused by a __precompile__(false) declaration in the module.
[ Info: Precompiling SpecialFunctionsChainRulesCoreExt [9eb7bdd4-e44c-55fc-b9cc-1a32cb715188] (cache misses: wrong dep version loaded (4))
┌ Warning: Module SpecialFunctions with build ID fafbfcf

1680×720×25 ImmersedBoundaryGrid{Float64, Bounded, Bounded, Bounded} on CPU with 7×7×7 halo:
├── immersed_boundary: GridFittedBottom(mean(z)=-256.434, min(z)=-2643.0, max(z)=0.0)
├── underlying_grid: 1680×720×25 LatitudeLongitudeGrid{Float64, Bounded, Bounded, Bounded} on CPU with 7×7×7 halo and with precomputed metrics
├── longitude: Bounded  λ ∈ [-126.998, -120.002] regularly spaced with Δλ=0.00416419
├── latitude:  Bounded  φ ∈ [47.0021, 49.9979]   regularly spaced with Δφ=0.00416088
└── z:         Bounded  z ∈ [-2643.0, 0.0]       regularly spaced with Δz=105.72

In [8]:
## plots the new bathymetry (this plot does not look right)
# using CairoMakie
# longitudes = range(long_min, long_max, length=Nx) #start:step:end --> first value, step size, last value
# latitudes = range(lat_min, lat_max, length=Ny)  

# #make a map of bathymetry
# fig = Figure()
# ax  = Axis(fig[1, 1],xlabel = "Longitude", ylabel = "Latitude", aspect = 0.6)
# hm = heatmap!(longitudes, latitudes, bottom_height, colorrange = (-250, 250), colormap = :bukavu)
# cb = Colorbar(fig[1,2], hm, label = "depth (m)")
# cb.height = Relative(1.0)
# # scatter!(236.5425 - 360, 48.24, color= :red, markersize = 10)

# display(fig)


In [9]:
# using CairoMakie #(better plot)
# fig = Figure()
# ax = Axis(fig[1,1], title= "bathymetry")
# hm = heatmap!(ax, bathymetry_interp, colormap=:topo, colorrange=(minimum(bathymetry_interp), maximum(bathymetry_interp)))
# display(fig)
# # save bathymetry if wanted
# # save("SalishSea_bathymetry1.png", fig)

In [10]:
# # check resolution / might need to check variable names
# using Statistics

# km_per_degree_lon = 111.32 * cosd(mean(latitudes))  
# km_per_degree_lat = 111.32 

# Δx_M = Δλ * km_per_degree_lon *100
# Δy_M = Δφ * km_per_degree_lat  *100

# @show Δx_M, Δy_M

In [9]:
#finds the nearest grid indices (x,y) corresponding with a lat and long
function find_nearest_index(grid, lon, lat, z)
    ugrid = grid.underlying_grid #determined using underlying grid when inspecting grid meta data 

    lon_vals = collect(ugrid.λᶜᵃᵃ)  # centered longitude
    lat_vals = collect(ugrid.φᵃᶜᵃ)  # centered latitude
    z_vals   = collect(ugrid.zᵃᵃᶜ)  # correct depth values
    z_idx = argmin(abs.(z_vals .- z))  


    lon_idx = argmin(abs.(lon_vals .- lon)) #subtracts each long value from desired lon value & finds the index of the closest value
    lat_idx = argmin(abs.(lat_vals .- lat))

    println("Nearest indices found: lon_idx=$lon_idx, lat_idx=$lat_idx")
    return lon_idx, lat_idx, z_idx
end

z = 0
const lat_release = 48.12   
const lon_release = -123.40 

lon_idx, lat_idx, z_idx = find_nearest_index(grid, lon_release, lat_release, z)

Nearest indices found: lon_idx=872, lat_idx=276


(872, 276, 32)

In [56]:
using Oceananigans
using Oceananigans.Fields: CenterField

#from luke's code

u₁₀ = 10    # Wind speed at 10m height (m/s)
ρₒ = 1026.0 # Ocean surface density (kg/m³)
cᴰ = 2.5e-3 # Drag coefficient (dimensionless)
ρₐ = 1.225  # Air density at sea level (kg/m³)

# Define wind stress forcing (momentum flux at surface)
@inline wind_stress_x(x, y, t, p=nothing) = (- ρₐ / ρₒ * cᴰ * u₁₀ * abs(u₁₀)) * rand()
@inline wind_stress_y(x, y, t, p=nothing) = (- ρₐ / ρₒ * cᴰ * u₁₀ * abs(u₁₀)) * rand()


# Boundary conditions applying wind stress
u_top_bcs = FluxBoundaryCondition(wind_stress_x) #Adds momentum input at the surface (east-west wind)
v_top_bcs = FluxBoundaryCondition(wind_stress_y) #Adds momentum input at the surface (north-south wind)

u_bcs = FieldBoundaryConditions(top = u_top_bcs) 
v_bcs = FieldBoundaryConditions(top = v_top_bcs)

c_bcs = FieldBoundaryConditions()  # Passive tracer boundary condition


@time model = HydrostaticFreeSurfaceModel(; 
    grid = grid,
    clock = Clock{Float64}(time = 0),
    momentum_advection = VectorInvariant(),  
    tracer_advection = Centered(),
    forcing = NamedTuple(), 
    boundary_conditions = (u=u_bcs, v=v_bcs, c=c_bcs),  
    tracers = (:c,), 
    pressure = nothing,
    diffusivity_fields = nothing,
    velocities = nothing, 
    auxiliary_fields = NamedTuple(),
)



ugrid = grid.underlying_grid

#note here: setting tracer to entire associated zC column for sole purpose of simulating -- change this 

r = size(model.tracers.c, 3)  #the max valid depth index

for k in 1:r  # Ensure we don't exceed depth range
    model.tracers.c[lon_idx, lat_idx, k] = 5.0
end



## set!(model, c=c_initial) redundant if using above code

  6.705333 seconds (7.25 M allocations: 7.975 GiB, 57.26% gc time)


In [57]:
u_initial = similar(model.velocities.u) 
v_initial = similar(model.velocities.v)
w_initial = similar(model.velocities.w)

u_initial .= 0.2  # test case for simulation -- change
v_initial .= 0.0  # no flow
w_initial .= 0.0  # no flow 

set!(model.velocities.u, u_initial)
set!(model.velocities.v, v_initial)
set!(model.velocities.w, w_initial)



1680×720×26 Field{Center, Center, Face} on ImmersedBoundaryGrid on CPU
├── grid: 1680×720×25 ImmersedBoundaryGrid{Float64, Bounded, Bounded, Bounded} on CPU with 7×7×7 halo
├── boundary conditions: FieldBoundaryConditions
│   └── west: ZeroFlux, east: ZeroFlux, south: ZeroFlux, north: ZeroFlux, bottom: Nothing, top: Nothing, immersed: ZeroFlux
└── data: 1694×734×40 OffsetArray(::Array{Float64, 3}, -6:1687, -6:727, -6:33) with eltype Float64 with indices -6:1687×-6:727×-6:33
    └── max=0.0, min=0.0, mean=0.0

In [62]:
# #T and S: skipping this for now because ECCO not retrieving data correctly 
#confirmed user/pass for ecco but still not working 


# T = ClimaOcean.ECCOMetadata(:temperature; dates) #retreiving metadata only 
# S = ClimaOcean.ECCOMetadata(:salinity; dates)
# # FT = ECCORestoring(T, CPU(); rate=1/2days)
# # FS = ECCORestoring(S, CPU(); rate=1/2days)
# set!(ocean.model, T = ClimaOcean.ECCOMetadata(:temperature; dates),
# S = ClimaOcean.ECCOMetadata(:salinity; dates))

In [17]:
# ugrid = grid.underlying_grid

# x_size = length(ugrid.λᶜᵃᵃ)  # Number of longitude points
# y_size = length(ugrid.φᵃᶜᵃ)  # Number of latitude points


In [18]:
# # `cᵃᵃᶜ` : where tracers will be stored ( cell centers )
# depth_vals = collect(ugrid.z.cᵃᵃᶜ)
# println("Depth range: ", (minimum(depth_vals), maximum(depth_vals)))
# println("Sample depth values: ", depth_vals[1:10])


In [20]:
# #keep this code for checks --> prints ugrid fields 
# ugrid = grid.underlying_grid  
# println("Available fields in ugrid: ", fieldnames(typeof(ugrid)))
# println(propertynames(ugrid.z))  # all available fields inside `z`, ugrid.z.cᵃᵃᶜ contains cell center z



# # # 'z' specific
# # println("Possible depth fields: ",
# #     filter(name -> occursin("z", string(name)), propertynames(ugrid))
# # 


In [22]:
# #this runs for 10 days & would be full implementation: have not tried this yet 
# using Oceananigans
# using Oceananigans.OutputWriters

# simulation = Simulation(model, Δt=10minutes, stop_time=10days)

# wizard = TimeStepWizard(cfl=0.3)
# simulation.callbacks[:wizard] = Callback(wizard, IterationInterval(10))

# simulation.output_writers[:tracer] = NetCDFOutputWriter(model, model.tracers,
#                               filename = "Tracer_2_19.nc",
#                               schedule = TimeInterval(30minutes))  

# simulation.callbacks[:flush_output] = Callback(TimeInterval(60minutes)) do sim
#     println("Flushing NetCDF output at time: ", sim.model.clock.time)
#     sim.output_writers[:tracer].flush()
# end

# println("Starting simulation with NetCDF output...")
# println("Simulation will run from 0 to ", simulation.stop_time)

# run!(simulation) 

# println("Simulation complete! Output saved to 'Tracer_2_19.nc'.")


In [45]:
# #tests 
# @show size(model.tracers.c)
# @show model.closure
# @show extrema(model.tracers.c)
# @show sum(model.tracers.c)

size(model.tracers.c) = (1680, 720, 25)


(1680, 720, 25)

In [60]:
#test case that I did run : 
using Oceananigans
using Oceananigans.OutputWriters

simulation = Simulation(model, Δt=10minutes, stop_time=60minutes) 

wizard = TimeStepWizard(cfl=0.2)
simulation.callbacks[:wizard] = Callback(wizard, IterationInterval(10))

simulation.output_writers[:output] = NetCDFOutputWriter(model, merge(model.tracers, model.velocities),
                              filename = "test4_output.nc", #update
                              schedule = TimeInterval(5minutes))
simulation.callbacks[:flush_output] = Callback(TimeInterval(60minutes)) do sim
    println("Checkpoint: NetCDF output written at time: ", sim.model.clock.time)
end

println("Starting short test run...")
println("Simulation will run for 1 hour of model time.")

run!(simulation) 

println("Test complete! Output saved to 'test_output.nc'.") #update

Starting short test run...
Simulation will run for 1 hour of model time.


[ Info: Initializing simulation...
[ Info:     ... simulation initialization complete (6.123 seconds)
[ Info: Executing initial time step...
[ Info:     ... initial time step complete (24.766 seconds).


LoadError: InterruptException: